In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun May 28 11:38:35 2023 

The MPC controller takes in the previous RTN state computed (not the measured 'x' variable)
"""

import math
import matplotlib.pyplot as plt
import numpy as np

from math import sqrt, sin, asin, cos, tan, atan2, pi
from numpy.linalg import norm, inv, pinv
from scipy.linalg import expm, sinm, cosm

from source import spacecraft
from aa273_control import *


In [2]:
##############################################################################
##############################################################################
###                                                                        ###
###              AUXILIARY FUNCTIONS FOR GPS CONSTELLATION                 ###
###                                                                        ###
##############################################################################
##############################################################################
def get_hill_frame(r, v):
    h = np.cross(r, v)  # Angular momentum vector
    r_hat = r / np.linalg.norm(r)  # Local X-axis (R)
    h_hat = h / np.linalg.norm(h)  # Local Z-axis (N)
    y_hat = np.cross(h_hat, r_hat)  # Local Y-axis (T)
    return np.array([r_hat, y_hat, h_hat])

def rtn_to_eci(target_ECI, chaser_RTN):
    r_target = target_ECI[0:3]
    v_target = target_ECI[3:6]
    r_chaser = chaser_RTN[0:3]
    v_chaser = chaser_RTN[3:6]
    matrix_rtn2eci = get_hill_frame(r_target, v_target)
    nuDot = norm(np.cross(r_target, v_target)) / (norm(r_target)**2)
    omega = np.array([0.0, 0.0, nuDot])
    rho = np.transpose(matrix_rtn2eci) @ r_chaser
    rhoDot = np.transpose(matrix_rtn2eci) @ (v_chaser + np.cross(omega, r_chaser))
    
    chaser_relative_ECI = np.zeros(6)
    chaser_relative_ECI[0:3] = rho
    chaser_relative_ECI[3:6] = rhoDot

    chaser_ECI = chaser_relative_ECI + target_ECI
    return chaser_ECI # Chaser in ECI frame

def eci_to_rtn(target_ECI, chaser_ECI):
    r_target = target_ECI[0:3]
    v_target = target_ECI[3:6]

    chaser_relative_ECI = chaser_ECI - target_ECI
    rho = chaser_relative_ECI[0:3]
    rhoDot = chaser_relative_ECI[3:6]

    nuDot = norm(np.cross(r_target, v_target)) / (norm(r_target)**2)
    omega = np.array([0.0, 0.0, nuDot])
    matrix_rtn2eci = get_hill_frame(r_target, v_target)

    r_chaser = matrix_rtn2eci @ rho
    v_chaser = matrix_rtn2eci @ rhoDot - np.cross(omega, r_chaser)

    chaser_RTN = np.zeros(6)
    chaser_RTN[0:3] = r_chaser
    chaser_RTN[3:6] = v_chaser

    return chaser_RTN # Chaser in RTN frame



In [3]:

# Function to convert ECEF (xyz) to Geodetic (lat-lon-alt) coordinates
def ecef_to_geodetic(pos):
    if len(pos) != 3:
        raise ValueError('ECEF to Geodetic: Position vector must be length 3!')
        return np.array([0,0,0])
    earthRad = 6378136.300       # Radius of Earth (WGS84)
    earthEcc = 0.081819190842622 # Earth eccentricity (WGS84)
    x, y, z, r = pos[0], pos[1], pos[2], norm(pos)
    lon = atan2(y,x)
    lat0 = asin(z/r) 
    lat = lat0 # Initial guess
    n, nMax = 0, 3 # Number of iterations
    error, tol = 1, 1E-12
    rxy = norm([x,y])
    while (error > tol) and (n < nMax):
        lat0 = lat
        N = earthRad / sqrt(1 - (earthEcc * sin(lat0))**2)
        lat = atan2(z + (N * earthEcc * earthEcc) * sin(lat0), rxy)
        error = abs(lat - lat0)
        n += 1
    alt = (rxy - cos(lat)) - N
    geodetic = np.array([lat, lon, alt])
    return geodetic

# Computes the azimuth, elevation, and range from positions 0 to 1
def compute_aer(pos0, pos1):
    lla = ecef_to_geodetic(pos0)
    lat, lon = lla[0], lla[1]
    so = sin(lon)
    co = cos(lon)
    sa = sin(lat)
    ca = cos(lat)
    rotation = np.array(
        [[-so,            co, 0.0],
         [-sa * co, -sa * so, ca ],
         [ ca * co,  ca * so, sa ]])
    enu = rotation @ (pos1 - pos0)
    azim = atan2(enu[0], enu[1]);
    elev = atan2(enu[2], norm(enu[0:2]));
    rnge = norm(enu);
    enu = np.array([azim, elev, rnge]);
    return enu


In [11]:

##############################################################################
##############################################################################
###                                                                        ###
###               SETTING UP OF THE ACTUAL FLIGHT SIMULATION               ###
###                                                                        ###
##############################################################################
##############################################################################
dt = 10
duration = 5000
timesteps = math.floor(duration / dt)

P_cost = 1e6*np.eye(6) # terminal state cost matrix
Q_cost = np.diag([1., 1, 1, 1e-3, 1e-3, 1e-3]) # state cost matrix
R_cost = 1e2*np.eye(3) # control cost matrix

N = 10 # MPC step horizon
max_iters = 200 # maximum number of iterations for SCP  
eps = 1e-1 # tolerance for SCP


#Define Spacecrafts
sc1_elements = [6918.14, 0.00001, 97.59760, 0.000000, -109.33800,  0.00827]
sc1 = spacecraft.Spacecraft( elements = sc1_elements )

target_sc_elements = [6918.14, 0.00001, 97.59760, 0.000000, -109.33800, -100]
sc_target = spacecraft.Spacecraft( elements = target_sc_elements )


sc1.forces['maneuvers'] = True

# Initialize the state history
chaser_ECI_history = np.zeros((timesteps, 6))
target_ECI_history = np.zeros((timesteps, 6))

chaser_RTN_history = np.zeros((timesteps, 6))

chaser_MPC_history = np.zeros((timesteps, N-1, 6))

thrust_history = np.zeros((timesteps, 3))

u_ECI = np.zeros(3)

In [12]:
##############################################################################
##############################################################################
###                                                                        ###
###                    ACTUAL FLIGHT SIMULATION BELOW                      ###
###                                                                        ###
##############################################################################
##############################################################################


# For loop, generate ground truth `xt` and noisy measurements `y`.
for k in range(timesteps):
    print('---Iteration: ', k)
    # Propagate the true states of all spacecraft using RK4 propagator
    sc1.set_thruster_acceleration(u_ECI/sc1.mass)
    sc1.propagate_perturbed(dt, dt)
    
    sc_target.propagate_orbit(dt)
    
    # Record states
    target_eci_k = np.array([sc_target.px, sc_target.py, sc_target.pz, sc_target.vx, sc_target.vy, sc_target.vz])
    chaser_eci_k = np.array([sc1.px, sc1.py, sc1.pz, sc1.vx, sc1.vy, sc1.vz])
    chaser_ECI_history[k, :] = chaser_eci_k
    target_ECI_history[k, :] = target_eci_k
    
    chaser_RTN = eci_to_rtn(target_eci_k, chaser_eci_k)
    chaser_RTN_history[k, :] = chaser_RTN

    # u_ECI = np.zeros(3)

    # Use MPC to compute the optimal control input
    # MPC begins at previous state (takes in RTN) and attempts to reach origin in RTN frame
    x_mpc, u_mpc = do_MPC(dt, sc1.n, sc1.mass, chaser_RTN, np.zeros(6), N, Q_cost, R_cost, P_cost, max_iters, eps)
    chaser_MPC_history[k, :, :] = x_mpc[2:, :]

    # Convert the MPC output to ECI
    u_ECI =  get_hill_frame(chaser_eci_k[0:3], chaser_eci_k[3:6]) @ u_mpc[1, :]
    thrust_history[k, :] = u_ECI
    print(u_mpc[1, :])
    print('u_ECI (N): ', u_ECI)
    print('u_ECI accel (km/s^2): ', u_ECI/sc1.mass)

    # print('x_mpc: ', x_mpc)
    # print('u_ECI: ', u_ECI)
    


---Iteration:  0
Converged in 115 iterationss
[9.95223054e+00 2.28166087e-01 3.91479192e-14]
u_ECI (N):  [-3.52435162 -1.19250201 -9.23340875]
u_ECI accel (km/s^2):  [-0.03524352 -0.01192502 -0.09233409]
---Iteration:  1
Converged in 149 iterationss
[10.06984557  0.21995477  2.1397169 ]
u_ECI (N):  [-3.51555335  2.12580259 -9.44194928]
u_ECI accel (km/s^2):  [-0.03515553  0.02125803 -0.09441949]
---Iteration:  2
Converged in 50 iterationss
[10.02328806  0.29230501  4.76814746]
u_ECI (N):  [-3.45148761  5.89582707 -8.75290994]
u_ECI accel (km/s^2):  [-0.03451488  0.05895827 -0.0875291 ]
---Iteration:  3
Converged in 97 iterationss
[9.98495262 0.29984807 7.87710797]
u_ECI (N):  [-3.23243151  9.83639315 -7.39153394]
u_ECI accel (km/s^2):  [-0.03232432  0.09836393 -0.07391534]
---Iteration:  4
Converged in 73 iterationss
[10.06549925  0.20619191 10.07189832]
u_ECI (N):  [-2.91315022 12.54895069 -6.06937544]
u_ECI accel (km/s^2):  [-0.0291315   0.12548951 -0.06069375]
---Iteration:  5


ValueError: math domain error

In [13]:
print(u_mpc)

print([np.linalg.norm(u_mpc[i], np.inf) for i in range(N-1)])

[[-1.04520579e-14 -1.04520579e-14 -3.48938421e-15]
 [ 1.00654992e+01  2.06191915e-01  1.00718983e+01]
 [ 9.97309679e+00 -1.23943464e-01  9.98451448e+00]
 [ 9.97483312e+00  6.36671101e-03  9.86418233e+00]
 [ 9.98089504e+00  1.38395898e-03 -3.87439954e-01]
 [ 1.00194432e+01  6.33324065e-04 -3.16283100e+00]
 [ 1.00310681e+01  2.08575580e-04 -8.86839741e-01]
 [ 1.00031333e+01 -2.35252738e-04  2.74018665e-01]
 [ 7.51270411e+00  1.15046689e-02  3.43056803e-02]
 [-9.98547799e+00 -9.32739110e-03 -1.23271545e-02]]
[1.045205790307591e-14, 10.07189832107689, 9.984514480460337, 9.974833117185202, 9.980895041427688, 10.019443168532826, 10.031068092653197, 10.003133266103662, 7.512704109788436]


In [ ]:
# ##############################################################################
# ##############################################################################
# ###                                                                        ###
# ###                               PLOTTING                                 ###
# ###                                                                        ###
# ##############################################################################
# ##############################################################################

# Plot the trajectory of the chaser and target in ECI frame
plt.figure()
plt.plot(chaser_ECI_history[1:, 0], chaser_ECI_history[1:, 1], 'bo', label='Chaser')
plt.plot(target_ECI_history[1:, 0], target_ECI_history[1:, 1], 'ro', label='Target')
plt.plot(chaser_ECI_history[1, 0], chaser_ECI_history[1, 1], '*', label='Chaser Initial')
plt.plot(chaser_ECI_history[-1, 0], chaser_ECI_history[-1, 1], '*', label='Chaser Final')
plt.plot(target_ECI_history[1, 0], target_ECI_history[1, 1], '*', label='Target Initial')
plt.xlabel('X [km]')
plt.ylabel('Y [km]')
plt.title('Trajectory of Chaser and Target in ECI frame')
plt.legend()

# Plot the trajectory of the chaser in RTN frame
plt.figure()
plt.plot(chaser_RTN_history[1:, 0], chaser_RTN_history[1:, 1], 'bo', label='Chaser')
plt.plot(chaser_RTN_history[1, 0], chaser_RTN_history[1, 1], '*', label='Chaser Initial')
plt.plot(chaser_RTN_history[-1, 0], chaser_RTN_history[-1, 1], '*', label='Chaser Final')
plt.plot(0, 0, 'x', label='Target')
plt.xlabel('R [km]')
plt.ylabel('T [km]')
plt.title('Trajectory of Chaser in RTN frame')
plt.legend()

# Plot thrust history
plt.figure()
plt.plot(thrust_history[:, 0], label='x')
plt.plot(thrust_history[:, 1], label='y')
plt.plot(thrust_history[:, 2], label='z')
plt.xlabel('Time [s]')
plt.ylabel('Thrust [N]')
plt.title('Thrust History')
plt.legend()

In [ ]:

# plt.close('all')


# # Convert km to m
# x_history  = x_history * 1000
# xtrue_history = xtrue_history * 1000
# target_true_history = target_true_history * 1000

# # Trajectory
# plt.figure()
# plt.title('Trajectory')
# q = 0
# plt.plot( xtrue_history[0, q:], xtrue_history[1, q:], 'b', label=' Chaser Truth')
# plt.plot( x_history[0, q:], x_history[1, q:], 'r', label=' Chaser Estimate')
# plt.plot(x_history[0, q], x_history[1, q], '*', label=' Chaser Start')
# plt.xlabel('X [km]')

# plt.plot(target_true_history[0,:], target_true_history[1,:], 'g', label=' Target Truth')
# plt.legend()

# plt.figure()


# # RTN Positions
# plt.title('RTN Positions')
# for t in range (29):
#     plt.plot(x_MPC_history[0, 0:4, t], x_MPC_history[1, 0:4, t], '--*', color='k')
# plt.plot(x_RTN_history[0,:], x_RTN_history[1,:], '-o', label=' Chaser Estimate')
# plt.plot(x_RTN_truth[0,:], x_RTN_truth[1,:], '-o', label=' Chaser Truth')
# plt.plot(x_RTN_history[0, q], x_RTN_history[1, q], '*', label=' Chaser Position')
# plt.plot(x_RTN_history[0, 0], x_RTN_history[1, 0], '*', label=' Chaser Start')
# plt.plot(0, 0, '*', label=' Target')

# plt.legend()

# plt.figure()
# plt.title('Control input')
# plt.plot(u_history[0,:], label='u_x')
# plt.plot(u_history[1,:], label='u_y')
# plt.plot(u_history[2,:], label='u_z')
# plt.xlim(0, 30)
# plt.ylabel('Force')
# plt.xlabel('Time steps (3000 sec)')
# plt.legend()

# #show plot
# plt.show()
